In [11]:
!pip install langchain openai google-search-results langchain-experimental
!pip install langchain-openai

In [30]:
!pip install langchain langchain-community google-search-results

In [36]:
from google.colab import userdata
import os

# Retrieve the API key from Colab secrets
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
SERPAPI_API_KEY = userdata.get('SERPAPI_API_KEY')

# Set it as an environment variable so the libraries can pick it up
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ['SERPAPI_API_KEY'] = SERPAPI_API_KEY

print("OPENAI_API_KEY loaded from Colab secrets and set as environment variable.")

OPENAI_API_KEY loaded from Colab secrets and set as environment variable.


###Chain of Thought

In [16]:


import os
from langchain_classic.chains import LLMChain
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate


api_key = OPENAI_API_KEY

prompt_template = "I went to the market and bought 20 mangoes. I gave 5 apples to the Ram and 3 to the House-help. I then went and bought 12 more mangoes and ate 2. How many mangoes did I remain with? Let's think step by step."

# Initializing LLM
llm = OpenAI(temperature=0, openai_api_key=api_key)

# Creating the Chain
llm_chain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(prompt_template))

# Running the chain
print(llm_chain.run({}))



1. Bought 20 mangoes
2. Gave 5 apples to Ram
3. Gave 3 apples to House-help
4. Bought 12 more mangoes
5. Ate 2 mangoes
6. Total mangoes bought = 20 + 12 = 32
7. Total mangoes given away = 5 + 3 = 8
8. Total mangoes eaten = 2
9. Remaining mangoes = 32 - 8 - 2 = 22
Therefore, I remained with 22 mangoes.


###Few-shot Prompting

In [14]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_openai import ChatOpenAI


# Defining examples for Few-Shot
examples = [
    {
        "Question": "Yes or no: Would a rubber duck sink in water?",
        "Reasoning": "The density of a typical rubber duck is less than water, typically around 0.3 g/cm^3. Therefore, a rubber duck would float.",
        "Answer": "No"
    },
    {
        "Question": "Yes or no: Would a wooden block sink in water?",
        "Reasoning": "The density of wood is generally less than water, averaging around 0.6 to 0.9 g/cm^3. Therefore, a wooden block would likely float.",
        "Answer": "No"
    },
    {
        "Question": "Yes or no: Would a plastic bottle filled with air sink in water?",
        "Reasoning": "The density of air is much less than water. If the plastic bottle is filled with air, its overall density would be lower than water, causing it to float.",
        "Answer": "No"
    },
    {
        "Question": "Yes or no: Would a metal coin sink in water?",
        "Reasoning": "Metals typically have higher densities than water. Therefore, a metal coin would likely sink in water.",
        "Answer": "Yes"
    },
    {
        "Question": "Yes or no: Would a helium-filled balloon sink in water?",
        "Reasoning": "Helium is less dense than air and a helium-filled balloon would be buoyant. It would rise in air and, in water, it would float.",
        "Answer": "No"
    }
]

# Template for the examples
example_prompt = PromptTemplate(
    input_variables=["Question", "Reasoning", "Answer"],
    template="Question: {Question}\nReasoning: {Reasoning}\nAnswer: {Answer}"
)

# Creating the FewShotPromptTemplate
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"]
)

api_key = OPENAI_API_KEY

# Running the chain
chain = LLMChain(llm=ChatOpenAI(openai_api_key=api_key), prompt=prompt)
print(chain.run("Yes or no: will plastic ball sink in water?"))

Answer: No


###REACT Reasoning

In [32]:
import os
from typing import Optional, Dict, Any
from langchain_community.utilities import SerpAPIWrapper
from langchain.tools import Tool

def build_serpapi_tool(
    api_key: Optional[str] = None,
    tool_name: str = "current_search",
    description: str = "A search engine. Useful for when you need to answer questions about current events. Input should be a search query.",
    params: Optional[Dict[str, Any]] = None
) -> Tool:
    """
    Constructs a LangChain Tool for SerpAPI web search.

    Args:
        api_key (Optional[str]): Your SerpAPI key. If None, looks for SERPAPI_API_KEY env variable.
        tool_name (str): The name the agent uses to refer to the tool.
        description (str): Instructions for the agent on when/how to use this tool.
        params (Optional[Dict]): Additional parameters for SerpAPI (e.g., {'gl': 'us', 'hl': 'en'}).

    Returns:
        Tool: A functional LangChain tool ready for agent binding.
    """

    # Check for API Key if not provided explicitly
    if not api_key:
        api_key = os.getenv("SERPAPI_API_KEY")
        if not api_key:
            raise ValueError("SerpAPI API key not found. Please provide it as an argument or set the SERPAPI_API_KEY environment variable.")

    # Initialize the Wrapper
    # params can be used to set location (gl), language (hl), or engine (e.g., google_scholar)
    search_wrapper = SerpAPIWrapper(
        serpapi_api_key=api_key,
        params=params if params else {}
    )

    # Construct the Tool
    return Tool(
        name=tool_name,
        func=search_wrapper.run,
        description=description
    )

In [37]:
web_search = build_serpapi_tool(api_key=SERPAPI_API_KEY)

In [39]:
llm = OpenAI(temperature=0)

web_search = build_serpapi_tool(api_key=SERPAPI_API_KEY)

react_agent = create_react_agent(
    model = "openai:gpt-4o",
    tools = [web_search],
    name = "react_agent",
)

result = react_agent.invoke({
    "messages": [
        {
            "role": "user",
            "content": "what's the combined headcount of the FAANG companies in 2024?"
        }
    ]
})

/tmp/ipython-input-206337428.py:20: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  react_agent = create_react_agent(


In [40]:
result

{'messages': [HumanMessage(content="what's the combined headcount of the FAANG companies in 2024?", additional_kwargs={}, response_metadata={}, id='86faffa8-380e-4173-8c21-8768becb0350'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 121, 'prompt_tokens': 83, 'total_tokens': 204, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_e413f45763', 'id': 'chatcmpl-CnTktcNDuMey1tMcVDpoWMsXiwmf3', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='react_agent', id='lc_run--019b2859-198a-7d10-b821-766a45d5840c-0', tool_calls=[{'name': 'current_search', 'args': {'__arg1': 'Facebook headcount 2024'}, 'id': 'call_oFfiO6QrEB3UwH8637OF0sVu', 'type': 'tool_cal

In [42]:
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
import json

def process_and_print_messages(message_history):
    """
    Parses a LangChain message history dictionary and prints it in a user-friendly format.

    Args:
        message_history (dict): A dictionary containing a list of message objects
                                (HumanMessage, AIMessage, ToolMessage).
    """
    messages = message_history.get('messages', [])

    print("\n" + "="*60)
    print("📋 CONVERSATION LOG")
    print("="*60 + "\n")

    for msg in messages:
        if isinstance(msg, HumanMessage):
            print(f"👤 **USER**: {msg.content}")
            print("-" * 60)

        elif isinstance(msg, AIMessage):
            # 1. Handle Tool Calls (The Agent "Thinking" & deciding to act)
            if msg.tool_calls:
                print(f"🧠 **AI THINKING** (Decided to call tools):")
                for tool in msg.tool_calls:
                    args = tool.get('args', {})
                    # Format arguments nicely
                    arg_str = ", ".join([f"{k}='{v}'" for k, v in args.items()])
                    print(f"   ▶ 🛠️  Using Tool: `{tool['name']}`")
                    print(f"     Input: {arg_str}")
                print("-" * 60)

            # 2. Handle Final Content (The Agent's actual response text)
            # Sometimes an AIMessage has both tool_calls and content, or just one.
            if msg.content:
                # If content is empty strings (common in intermediate steps), skip printing headers
                if msg.content.strip():
                    print(f"🤖 **AI RESPONSE**:\n{msg.content}")
                    print("-" * 60)

        elif isinstance(msg, ToolMessage):
            # Clean up tool output. If it's a long list/JSON string, truncate it for readability.
            content = msg.content
            if len(content) > 300:
                content_preview = content[:300] + "... [truncated]"
            else:
                content_preview = content

            print(f"⚙️ **TOOL OUTPUT** (Source: {msg.name}):")
            print(f"   📄 Result: {content_preview}")
            print("-" * 60)

    print("\n" + "="*60 + "\n")


In [43]:
process_and_print_messages(result)


📋 CONVERSATION LOG

👤 **USER**: what's the combined headcount of the FAANG companies in 2024?
------------------------------------------------------------
🧠 **AI THINKING** (Decided to call tools):
   ▶ 🛠️  Using Tool: `current_search`
     Input: __arg1='Facebook headcount 2024'
   ▶ 🛠️  Using Tool: `current_search`
     Input: __arg1='Amazon headcount 2024'
   ▶ 🛠️  Using Tool: `current_search`
     Input: __arg1='Apple headcount 2024'
   ▶ 🛠️  Using Tool: `current_search`
     Input: __arg1='Netflix headcount 2024'
   ▶ 🛠️  Using Tool: `current_search`
     Input: __arg1='Google headcount 2024'
------------------------------------------------------------
⚙️ **TOOL OUTPUT** (Source: current_search):
   📄 Result: ['Meta Platforms total employee count in 2024 was 74,067, a 10.03% increase from 2023. Meta Platforms total number of employees in 2023 was 67,317, a 22.16% ...', 'In 2022, Meta Platforms reached a record number of full time employees, with over 86 thousand people working fo

###Tree-of-Thought

In [49]:
from langchain_experimental.smart_llm import SmartLLMChain

# 1. Setup API Key
api_key = OPENAI_API_KEY

# 2. Define the Logic Puzzle (The "Hard Question")
# The problem: Measure 6 liters using 12L, 4L, and 3L jugs.
hard_question = "I have a 12 liter jug, a 4 liter jug and a 3 liter jug. I want to measure 6 liters. How do I do it?"

# 3. Create the Prompt Template
prompt = PromptTemplate.from_template(hard_question)

# 4. Initialize the LLM
# Temperature is set to 0 to encourage precise reasoning.
llm = ChatOpenAI(temperature=0, openai_api_key=api_key)

# 5. Initialize the SmartLLMChain
# n_ideas=3 instructs the model to generate 3 diverse initial approaches (branches of the tree).
# verbose=True allows you to see the "thought process," critiques, and final selection in the output.
chain = SmartLLMChain(llm=llm, prompt=prompt, n_ideas=3, verbose=True)

# 6. Run the Chain
chain.run({})



> Entering new SmartLLMChain chain...
Prompt after formatting:
I have a 12 liter jug, a 4 liter jug and a 3 liter jug. I want to measure 6 liters. How do I do it?
Idea 1:
1. Fill the 4 liter jug with water.
2. Pour the water from the 4 liter jug into the 12 liter jug.
3. Fill the 4 liter jug again with water.
4. Pour the water from the 4 liter jug into the 12 liter jug until it is full (which will leave 2 liters in the 4 liter jug).
5. Empty the 12 liter jug.
6. Pour the remaining 2 liters from the 4 liter jug into the 12 liter jug.
7. Fill the 4 liter jug again with water.
8. Pour the water from the 4 liter jug into the 12 liter jug until it is full (which will leave 6 liters in the 4 liter jug).

Therefore, you have successfully measured 6 liters using the 12 liter, 4 liter, and 3 liter jugs.
Idea 2:
1. Fill the 4 liter jug with water.
2. Pour the water from the 4 liter jug into the 12 liter jug.
3. Fill the 4 liter jug again with water.
4. Pour the water from the 4 liter jug into 

'Idea 1: \n1. Fill the 4 liter jug with water.\n2. Pour the water from the 4 liter jug into the 12 liter jug.\n3. Fill the 4 liter jug again with water.\n4. Pour the water from the 4 liter jug into the 12 liter jug until it is full (which will leave 2 liters in the 4 liter jug).\n5. Empty the 12 liter jug.\n6. Pour the remaining 2 liters from the 4 liter jug into the 12 liter jug.\n7. Fill the 4 liter jug again with water.\n8. Pour the water from the 4 liter jug into the 12 liter jug until it is full (which will leave 6 liters in the 4 liter jug).\n\nImproved Answer:\n1. Fill the 4 liter jug with water.\n2. Pour the water from the 4 liter jug into the 12 liter jug.\n3. Fill the 4 liter jug again with water.\n4. Pour the water from the 4 liter jug into the 12 liter jug until it is full (which will leave 2 liters in the 4 liter jug).\n5. Empty the 12 liter jug.\n6. Pour the remaining 2 liters from the 4 liter jug into the 12 liter jug.\n7. Fill the 4 liter jug again with water.\n8. Pour 